In [2]:
import pandas as pd
import numpy as np

In [4]:
russell_df = pd.read_csv("russell2000pvdata.csv", error_bad_lines = False)
vars = ['open', 'high', 'low', 'close', 'volume']
rawdata = {}
period = 20           #NUMBER OF TRADING DAYS TO COMPUTE MEAN REVERSION OVER.  THIS IS A PARAMETER FOR FITTING
universesize = 2000                 #SIZE OF PORTFOLIO IN NUMBER OF STOCKS.  IF THIS IS MORE THAN 2000, IT WILL GENERALLY JUST BE CAPPED AT 2000 SINCE BASE UNIVERSE IS RUSSELL 2000
maxindividualweight = 0.01          #MAXIMUM FRACTION A SINGLE POSITION CAN TAKE UP OF ENTIRE PORTFOLIO.  0.01 MEANS 1%.  i.e. if you have a portfolio of $100 million, max single position size is $1 million

b'Skipping line 109845: expected 8 fields, saw 11\n'


In [5]:
for tvar in vars:
    rawdata[tvar] = russell_df.loc[:, ['tickerid', 'ticker', 'date', tvar]]
    rawdata[tvar] = rawdata[tvar].pivot(index = 'date', columns = 'ticker', values = tvar)
    rawdata[tvar] = rawdata[tvar].iloc[:, :universesize]

In [14]:
closing_data = rawdata['close']
closing_data.dropna(axis=1, inplace=True)
closing_data

ticker,AADR,AAON,AAWW,ABCB,ABEO,ABG,ABM,ABR,ACCO,ACIW,...,X,XOMA,XPER,YORW,ZEUS,ZGNX,ZIOP,ZIXI,ZUMZ,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
20101227,29.590000,8.586667,57.250000,10.920000,129.00,18.100000,26.760000,5.99,8.75,9.066667,...,58.070000,122.800003,22.309999,17.730000,27.940001,47.759998,4.83,4.49,28.830000,0.55
20101228,29.590000,8.524444,56.500000,10.960000,124.50,17.820000,26.790001,5.95,8.71,9.133333,...,58.279999,95.000000,22.260000,17.750000,27.719999,45.119999,4.71,4.26,28.650000,0.55
20101229,29.790001,8.548148,56.810001,11.070000,123.50,18.450001,26.730000,6.00,8.75,9.203333,...,58.509998,102.000000,22.340000,17.580000,28.469999,44.480000,4.75,4.28,28.209999,0.58
20101230,29.730000,8.548148,56.490002,10.740000,122.50,18.799999,26.600000,5.88,8.77,9.146667,...,59.020000,106.599998,22.100000,17.309999,28.860001,46.400002,4.77,4.31,27.709999,0.62
20101231,29.840000,8.358519,55.830002,10.540000,124.00,18.480000,26.299999,5.96,8.52,8.956667,...,58.419998,102.599998,22.150000,17.290001,28.680000,45.360001,4.66,4.27,26.870001,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201217,59.939999,64.839996,56.000000,38.590000,1.60,147.649994,40.610001,14.73,9.02,38.990002,...,18.559999,41.439999,19.980000,47.689999,15.200000,20.299999,2.97,7.53,38.410000,14.25
20201218,60.779999,64.320000,55.270000,37.799999,1.54,142.600006,39.930000,14.21,8.95,38.540001,...,16.870001,45.560001,20.440001,47.259998,13.950000,19.990000,2.89,8.41,37.910000,14.00
20201221,60.049999,64.239998,55.389999,36.669998,1.55,141.460007,38.700001,13.79,8.65,38.090000,...,17.379999,45.610001,20.040001,46.020000,13.470000,20.209999,2.80,8.33,36.630001,13.94


In [22]:
ema_short = closing_data.ewm(span=50, adjust=False).mean()
ema_long = closing_data.ewm(span=200, adjust=False).mean()

In [23]:
cross = np.where(ema_short > ema_long, 'True', 'False')
ticker = closing_data.columns.values.tolist()
date = list(closing_data.index.values)

In [24]:
cross = pd.DataFrame(cross, columns = ticker, index = date)
cross

,AADR,AAON,AAWW,ABCB,ABEO,ABG,ABM,ABR,ACCO,ACIW,...,X,XOMA,XPER,YORW,ZEUS,ZGNX,ZIOP,ZIXI,ZUMZ,ZYXI
20101227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20101228,False,False,False,True,False,False,True,False,False,True,...,True,False,False,True,False,False,False,False,False,False
20101229,True,False,False,True,False,True,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True
20101230,True,False,False,False,False,True,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True
20101231,True,False,False,False,False,True,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201217,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,False,True,True,False
20201218,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,False,True,True,False
20201221,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,False,True,True,False
20201222,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,False,False,True,True,False
